In [5]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [9]:
def centroid_fire(fireID, year = '2023', path_region = "BOREAL_NRT_3571_DPS" ):
    fr = fv.load_large_fire(fireID, year = year, path_region= path_region, s3_path = True)
    fr = fr[fr.t == fr.t.max()]
    tmp = pd.DataFrame([[fr.lat.iloc[0], fr.lon.iloc[0], fr.farea.iloc[0], path_region, fireID]], columns= ["lat", "lon", "farea", "data_source", "fireID"])
    return(tmp)

# def concat_subsets(files):
#     df = []
#     for f in files:
#         manyfr = pd.read_csv(f)

#         manyfr = gpd.GeoDataFrame(manyfr)

#         manyfr.t = manyfr.t.astype("datetime64[ns]")
#         df.append(manyfr)
#     df = pd.concat(df)
#     return(df)

def concat_subsets(files):
    df = []
    for f in files:
        #manyfr = pd.read_csv(f, parse_dates=['t'], date_parser=dateparse)
        manyfr = pd.read_csv(f)
        manyfr = gpd.GeoDataFrame(manyfr)
        
        
        manyfr.t[~manyfr.t.str.contains("12:00:00")] = manyfr.t[~manyfr.t.str.contains("12:00:00")] + " 00:00:00"
        #print(manyfr.t[~manyfr.t.str.contains("12:00:00")])
        
        manyfr.t = manyfr.t.astype("datetime64[ns]")
        #manyfr.t = manyfr.t.astype("str")

        
        #manyfr.t.tz("UCT")
        df.append(manyfr)
    df = pd.concat(df)
    #df.t = df.t.astype("datetime64[ns]")
    return(df)

In [14]:
 
#fr = fv.load_large_fire('4049', year = '2023', path_region= 'backup_BOREAL_NRT_3571_DPS_largefire') #

In [13]:
#centroid_fire('4049')

['/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230430AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230430PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230501AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230501PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230502AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230502PM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/F4049_20230503AM', '/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:584: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:585: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,lat,lon,farea
0,53.659499,-115.145972,36.93166


In [7]:
#

from datetime import date
import os

year = None
regnm = 'BOREAL_NRT_3571_DPS'


def lf_ids(year = None, regnm = 'BOREAL_NRT_3571_DPS'):
    
    diroutdata = "s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/"

    if year == None:
        year = date.today().year

    if diroutdata.startswith("s3://"):
        # Can't use glob for S3. Use s3.ls instead.
        import s3fs
        s3 = s3fs.S3FileSystem(anon=False)
        s3path = os.path.join(diroutdata, regnm, str(year), "Largefire")
        fnms = [f for f in s3.ls(s3path)]


    fnms.sort()
    ids = []
    for f in fnms:
        fnm_lts = os.path.basename(f) 
        one_id = fnm_lts[1:-11]
        ids.append(one_id)
    tmp_ids = pd.DataFrame(ids, columns=["ids"])
    tmp_ids = tmp_ids.ids.unique()
    return(tmp_ids)


def unique(list1):
 
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return(unique_list)

def get_listed_ids(quebec_stats):
    newlist = [x.strip('][\n').split(' ') for x in quebec_stats.fireID.unique()]
    newlist = list(chain(*newlist))
    newlist = [x.replace('\n', ' ') for x in newlist]
    newlist = unique(newlist)
    return(newlist)

In [49]:

#/projects/shared-buckets/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire/

's3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-s3-conus/backup_BOREAL_NRT_3571_DPS_largefire/2023/Largefire'

In [50]:
#fnm_lts = os.path.basename(f)

In [8]:
#fnm_lts

In [51]:
ids = lf_ids()

files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats//boreal_snapstats*.csv")

fire_stats = concat_subsets(files)

#fire_stats.t.max()

### Subsetting fire stats by largefire record, so don't waste time looking for IDs that we haven't got yet. Wait, not worth it, size a bigger thing anyway.
#fire_stats = fire_stats[fire_stats.t < "2023-07-20 12:00:00"]

#fire_stats = pd.read_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats/boreal_snapstats_20231024.csv")

quebec_stats = fire_stats[fire_stats.prov_name_en == "Quebec"]

tmp_list = get_listed_ids(quebec_stats)

tmp_list = list(set(tmp_list).intersection(ids))

In [52]:
#fv.load_large_fire('10009', year = '2023', path_region = "BOREAL_NRT_3571_DPS", s3_path= True)

In [53]:
#fires = pd.DataFrame([[None, None, None, None]], columns= ["lat", "lon", "farea", "data_source"])
fires = []
for n,i in enumerate(tmp_list, start = 0):
    try:
        foo = centroid_fire(i)

    except Exception as e:
        print("Error at ID: ",i, e)
        continue

    fires.append([foo.lat.iloc[0], foo.lon.iloc[0], foo.farea.iloc[0], foo.data_source.iloc[0], foo.fireID.iloc[0]])
    #print(fires)
    if((n%5 == 0) | (n == (len(ids) -1))):
        print(i)
        fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source", "fireID"])
        fr_pd.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/lf_centroids/"+"fire_centroids_Quebec_from_boreal_snapstats" + regnm +".csv")
   

In [12]:
### Outputting a new list of centroids based on IDS that I saw were "missing" from the intersection of LF ids and snapshot IDS. It's totally possible that these IDS were not in LF form by the time I did this extraction back in november. 

# Get IDs. These IDs come from csvs made by old_shared/fire_weather_vis/Lightning_analysis/snap_prov_lightning.ipynb
# by going through the snapshot files, doing a spatial join, and collecting IDs. 

files = glob.glob("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats//boreal_snapstats*.csv")


fire_stats = concat_subsets(files)

fire_stats.t.max()

### Subsetting fire stats by largefire record, so don't waste time looking for IDs that we haven't got yet. Wait, not worth it, size a bigger thing anyway.
#fire_stats = fire_stats[fire_stats.t < "2023-07-20 12:00:00"]

#fire_stats = pd.read_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/snap_stats/boreal_snapstats_20231024.csv")

quebec_stats = fire_stats[fire_stats.prov_name_en == "Quebec"]

tmp_list = get_listed_ids(quebec_stats)

ids_lf = lf_ids( year = "2023")

tmp_list = list(set(tmp_list).intersection(ids_lf))

# Check what IDS are missing an re-run

path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/againQuebec_only_GRIDDED_20_days_BOREAL_NRT_3571_DPSgridded20240122.csv" ## looking for lightning 20 days before start
tmp_fires = fv.prep_fire_files(path)

missed_ids = set(tmp_list).symmetric_difference(tmp_fires.fireID.unique())

ids = missed_ids
fires = []
for n,i in enumerate(ids, start = 0):
    try:
        foo = centroid_fire(i)

    except Exception as e:
        print("Error at ID: ",i, e)
        continue

    fires.append([foo.lat.iloc[0], foo.lon.iloc[0], foo.farea.iloc[0], foo.data_source.iloc[0], foo.fireID.iloc[0]])
    #print(fires)
    fr_pd = pd.DataFrame(fires, columns=["lat", "lon", "farea", "data_source", "fireID"])
    fr_pd.to_csv("/projects/old_shared/fire_weather_vis/Lightning_analysis/lf_centroids/"+  "Generate_missing_lf" +"fire_centroids_Quebec_from_boreal_snapstats"  +".csv")
    

/tmp/ipykernel_1465/609185162.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manyfr.t[~manyfr.t.str.contains("12:00:00")] = manyfr.t[~manyfr.t.str.contains("12:00:00")] + " 00:00:00"
/tmp/ipykernel_1465/609185162.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manyfr.t[~manyfr.t.str.contains("12:00:00")] = manyfr.t[~manyfr.t.str.contains("12:00:00")] + " 00:00:00"
/tmp/ipykernel_1465/609185162.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

['F16417']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['F16441']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


...

/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['F16046']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


['F16143']


/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:595: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lon'] = gdf.centroid.x
/projects/old_shared/fire_weather_vis/base-fwi-vis/fwiVis/fwiVis.py:596: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


In [92]:
fr_pd = pd.DataFrame(fires)
fr_pd

,0,1,2,3,4
0,lat lon farea ...,lat lon farea ...,lat lon farea ...,lat lon farea ...,lat lon farea ...


In [84]:
pd.DataFrame([fires[1], fires[0]])

ValueError: Must pass 2-d input. shape=(2, 1, 4)